spark-submit --master yarn --deploy-mode cluster  --conf spark.yarn.appMasterEnv.SPARK_HOME=/share/apps/spark/^Cark-2.4.0-bin-hadoop2.6 --conf spark.yarn.submit.waitAppCompletion=false --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.speculation=false --conf spark.executorEnv.LANG=en_US.UTF-8 --conf spark.yarn.appMasterEnv.LANG=en_US.UTF-8 --driver-cores 20 --driver-memory 40G --num-executors 40 --executor-cores 15 --executor-memory 40G ./covid/py/get-summary-statistics.py

In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,hour,dayofmonth,dayofweek,to_timestamp,size,isnan,lit,date_format,to_timestamp,struct
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType, DoubleType

In [2]:
try:
    spark
except NameError:
    spark=SparkSession.builder.appName("").getOrCreate()

In [13]:
source='cuebiq'
country='ID'
n_chunks=1
start_date='2020-01-01'
end_date=datetime.today().strftime('%Y-%m-%d')
directories=[x.strftime('%Y-%m-%d').replace('-','')+'00' for x in pd.date_range(start_date,end_date)]
fs=spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())

if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/covid/data'
    directories=directories[:1]
else:
    path_to_data='/user/spf248/covid/data'
    
paths=[]
for directory in directories:
    path_to_directory=os.path.join(path_to_data,source,'s3',country,directory)
    if not fs.exists(spark._jvm.org.apache.hadoop.fs.Path(path_to_directory)):
        continue
    list_status=fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path_to_directory))
    paths.extend([file.getPath().toString().replace('hdfs://dumbo','').replace('file:','') for file in list_status])
    paths=sorted([path for path in paths if '.csv.gz' in path])
    print(directory)
    
if os.getenv('CLUSTER')=='PRINCE':
    paths=paths[:1]
    paths_chunks=np.array_split(paths,n_chunks)
    paths_chunks=paths_chunks[:1]
else:
    paths_chunks=np.array_split(paths,n_chunks)

print('# Files:', sum([len(paths_chunk) for paths_chunk in paths_chunks]))
print('# Chunks:', len(paths_chunks))
    
schema= StructType([
StructField("_c0", FloatType(), False),
StructField("_c1", StringType(), False),
StructField("_c2", FloatType(), False),
StructField("_c3", FloatType(), False),
StructField("_c4", FloatType(), False),
StructField("_c5", FloatType(), False),
StructField("_c6", FloatType(), False),
StructField("_c7", StringType(), False),
StructField("_c8", StringType(), False),])

2020010100
# Files: 1
# Chunks: 1


In [4]:
def load_data(paths_chunk):

    df=spark.read.option(
    'compression', 'gzip').option(
    'header', 'false').option(
    "multiLine", "true").option(
    'escape','"').option(
    "encoding", "UTF-8").option(
    "delimiter", "\t").schema(schema).csv(list(paths_chunk))

    column_names=[
    'timestamp',
    'cuebiq_id',
    'device_type',
    'latitude',
    'longitude',
    'accuracy',
    'time_zone_offset',
    'classification_type',
    'transformation_type']
    df=df.toDF(*column_names)

    df=df.withColumn(
    "time",to_timestamp(df["timestamp"]+df["time_zone_offset"])).withColumn(
    "date", date_format(col("time"), "yyyy-MM-dd")).withColumn(
    'dayofweek',date_format("time","u")).withColumn('hour',hour("time")).withColumn(
    'point', struct('longitude','latitude'))
    
    return df.select('cuebiq_id','device_type','time','date','dayofweek','hour','point','classification_type')

In [5]:
for i,paths_chunk in enumerate(paths_chunks):
    
    df=load_data(paths_chunk)
    df.cache()
    
    if not i:
        
        n_pings_id=df.groupby('cuebiq_id').agg(
        {'device_type':'first'}).withColumnRenamed('first(device_type)','device_type')
        
        n_pings_id_date=df.groupby('cuebiq_id','date').count().withColumnRenamed('count','n_pings')
        
        n_pings_id_personal=df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point').count().withColumnRenamed('count','n_pings')
            
        n_pings_id_personal_date=df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point','date').count().withColumnRenamed('count','n_pings')
        
        n_pings_id_personal_day_hour=df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point','dayofweek','hour').count().withColumnRenamed('count','n_pings')
        
        n_pings_id_day_hour=df.groupby(
        'cuebiq_id','dayofweek','hour').count().withColumnRenamed('count','n_pings')
    else:
        
        n_pings_id=n_pings_id.unionByName(
        df.groupby('cuebiq_id').agg({'device_type':'first'}).withColumnRenamed('first(device_type)','device_type'))
        
        n_pings_id_date=n_pings_id_date.unionByName(
        df.groupby('cuebiq_id','date').count().withColumnRenamed('count','n_pings'))
        
        n_pings_id_personal=n_pings_id_personal.unionByName(
        df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point').count().withColumnRenamed('count','n_pings'))
        
        n_pings_id_personal_date=n_pings_id_personal_date.unionByName(
        df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point','date').count().withColumnRenamed('count','n_pings'))
        
        n_pings_id_personal_day_hour=n_pings_id_personal_day_hour.unionByName(
        df.filter(df['classification_type']=='PERSONAL_AREA').groupby(
        'cuebiq_id','point','dayofweek','hour').count().withColumnRenamed('count','n_pings'))
        
        n_pings_id_day_hour=n_pings_id_day_hour.unionByName(df.groupby(
        'cuebiq_id','dayofweek','hour').count().withColumnRenamed('count','n_pings'))
        
    df.unpersist()

In [6]:
n_pings_id=n_pings_id.groupby('cuebiq_id').agg({'device_type':'first'}).withColumnRenamed('first(device_type)','device_type')
n_pings_id_date=n_pings_id_date.groupby('cuebiq_id','date').agg({'n_pings':'sum'}).withColumnRenamed('sum(n_pings)','n_pings')
n_pings_id_personal=n_pings_id_personal.groupby('cuebiq_id','point').agg({'n_pings':'sum'}).withColumnRenamed('sum(n_pings)','n_pings')
n_pings_id_personal_date=n_pings_id_personal_date.groupby('cuebiq_id','point','date').agg({'n_pings':'sum'}).withColumnRenamed('sum(n_pings)','n_pings')
n_pings_id_personal_day_hour=n_pings_id_personal_day_hour.groupby('cuebiq_id','point','dayofweek','hour').agg({'n_pings':'sum'}).withColumnRenamed('sum(n_pings)','n_pings')
n_pings_id_day_hour=n_pings_id_day_hour.groupby('cuebiq_id','dayofweek','hour').agg({'n_pings':'sum'}).withColumnRenamed('sum(n_pings)','n_pings')

In [7]:
n_pings_id_date.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'n_pings_id_date'))
n_pings_id_personal.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'n_pings_id_personal'))
n_pings_id_personal_date.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'n_pings_id_personal_date'))
n_pings_id_personal_day_hour.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'n_pings_id_personal_day_hour'))
n_pings_id_day_hour.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'n_pings_id_day_hour'))

In [8]:
users=(n_pings_id_date.groupby('cuebiq_id').agg(
{'date':'count','n_pings':'sum'}).withColumnRenamed(
'count(date)','n_days').withColumnRenamed('sum(n_pings)','n_pings')).join(
n_pings_id_personal.groupby('cuebiq_id').count().withColumnRenamed(
'count','n_personal'),on=['cuebiq_id']).join(n_pings_id,on=['cuebiq_id'])
users.write.mode("overwrite").parquet(os.path.join(path_to_data,source,'aggregates',country,'users_profile'))

In [16]:
users.show()

+--------------------+------+-------+----------+-----------+
|           cuebiq_id|n_days|n_pings|n_personal|device_type|
+--------------------+------+-------+----------+-----------+
|5476c2d50fa3396a6...|     3|     17|         1|        0.0|
|a28a817158e80e35d...|     2|     36|         1|        0.0|
|39d3c113a30f1fd7a...|     2|    179|         1|        1.0|
|ba5844efcdf7b415a...|     2|    135|         1|        1.0|
|2b3f9d1040ff00bef...|     2|    381|         1|        1.0|
|80212d453ce93ce85...|     2|    662|         2|        1.0|
|b2e54fa9d2420ed40...|     2|     80|         2|        1.0|
|c05b4cf1328f36cce...|     2|    234|         2|        0.0|
|a63778ecb57bbe24c...|     2|     56|         1|        1.0|
|8eb9cdc18ed67ce85...|     2|     29|         2|        0.0|
|21f619f1cc118d432...|     2|    185|         2|        1.0|
|20132c25e188c1543...|     2|    159|         1|        1.0|
|3415decfb70d9cb42...|     2|     37|         1|        0.0|
|0e59242866089fc17...|  